# SCRAP SCORES & FIXTURES FOR A COUNTRY FOR SPECIFIC SEASON

#### DATE- TIME - COMP (ALL ) - OPPONENT - MATCH DETAILS GENERAL

##### SAVED IN SCORES&FIXTURES_ALLCOMP_TEAM#####

In [1]:
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import csv
import time
from csv import DictWriter

DOMAIN="https://fbref.com"
URL = 'https://fbref.com/en/countries/'

response = rq.get(URL)

def get_soup(url):
    return bs(rq.get(url).text,"html.parser")

soup=get_soup(URL)


In [2]:
def findtableworlcupmen(link):
    if 'href' in link.attrs and link.text=='Men': #just Men
        #print(str(DOMAIN+link.attrs['href'])+"\n")
        URLmen=DOMAIN+link.attrs['href']
        #go to this page
        response = rq.get(URLmen)
        #print(URLmen)
        soup=get_soup(URLmen) #page html du pays
        table=soup.find('table',{'id':'comps_cup_1'})
        return table

In [3]:
def findcountryandlinks(tr):
    if tr.find('strong'):
        country=tr.find('strong') #pays a enregistrer dans pandas
        country=country.text
        if country>="Po":
            tds=tr.find('td',{"data-stat":"national_teams"})
            #print(tds)
            links=tds.find_all('a')
            return country,links
        else:
            return None
    else:
        return None

In [4]:
def findsquad(row):
    thannee=row.find('th') #annee a enregistrer dans pandas
    annee=thannee.text
    #print(thannee.text)
    tdsannee=row.find('td',{'data-stat':"squad"})
    return annee,tdsannee

In [5]:

def savetablecsv(table,country,annee):
    rows = table.find_all("tr")
    
    file= "Scores&Fixtures_AllComp_Team/" + country + "_" + annee + "_Scores&Fixtures.csv"
    opened=False
    with open(file, "wt+", encoding="utf-8",newline="") as f:
        writer = csv.writer(f)
        
        
        for row in rows:
            csv_row = []
            datematch=None
            tdlinkmatch=None
            dict_infos_match={}
            for cell in row.findAll(["td", "th"]):
                tdlinkmatch=row.find('td',{"data-stat":"match_report"})
                date=row.find('th',{"data-stat":"date"})
                opponent=row.find('td',{"data-stat":"opponent"})
                #print(tdlinkmatch)
                if tdlinkmatch!=None:
                    dict_infos_match=matchreportpage(tdlinkmatch,date,opponent,country) #dict with infos from match we add it to column in file

                csv_row.append(cell.get_text())
                
            writer.writerow(csv_row)
            if (dict_infos_match):
                writefilematch(dict_infos_match,country) #send opponent and date to save in file


In [6]:
def writefilematch(dict_infos_match,country):
    fieldnames=['Date','Country','Opponent','Manager1','Captain1','Manager2','Captain2','VenueName']
    try:
        date=dict_infos_match.get('Date')
    except:
        date="DateInconnu"
    try:
        opponent=dict_infos_match.get('Opponent')
    except:
        opponent="Inconnu"

    #save in MatchesDetails
    if opponent!=None and date!=None:
        file= "MatchesDetails/" + country + "_" + opponent + "_" + date + "_MatchDetails.csv" 
        with open(file, 'w', newline='',encoding="utf-8") as f:
            writer = csv.writer(f)
            writer = csv.DictWriter(f, fieldnames=fieldnames, dialect='excel')
            writer.writeheader()
            writer.writerow(dict_infos_match)

In [7]:
def matchreport(soupmatch,dateexacte,opponentname,country):
    dict_infos_match={}
    teams=[]
    dict_infos_match['Date']=dateexacte
    dict_infos_match['Country']=country
    dict_infos_match['Opponent']=opponentname
    
    scorebox=soupmatch.find('div',{'class':'scorebox'})
    if scorebox!=None:
        divsTeams=scorebox.find_all('div',{'itemprop':"performer"})
        for divt in divsTeams:
            teams.append(divt.strong.text) #Alg Belg to know which manager is for which team after
            
        divs= scorebox.find_all('div',{'class':"datapoint"})
        i=0
        nom1=(teams[0]) #Algeria
        nom2=(teams[1]) #Rwanda
        for div in divs:
            #add strong value
            job=(div.strong.text) #Manager
            name=div.text
            nom=(name.split(':'))[1] #name 
            if(i==0 or i==1):
                job=(job + "" + "1")
            else:
                job=(job + "" + "2")
            job=job.replace('\n', "") 
                
            dict_infos_match[job]=nom
            i+=1
            
        #add venue also
        divmeta=scorebox.find('div',{'class':'scorebox_meta'})
        try:
            venue=divmeta.find_all('small')[3].get_text()
            dict_infos_match['VenueName']=venue
        except:
            dict_infos_match['VenueName']="Inconnu"

        return dict_infos_match

In [8]:
def matchreportpage(tdlinkmatch,date,opponent,country):
     #take a href 
     #DOMAIN+"en/matches/16253d86/Belarus-France-September-6-2016-WCQ----UEFA-M"
    if 'href' in tdlinkmatch.find('a').attrs:
        URLmatch=DOMAIN+tdlinkmatch.find('a').attrs['href']
        #print(URLmatch)
        #go to this page
        response = rq.get(URLmatch)
        #print(URLmen)
        soupmatch=get_soup(URLmatch) #page html du pays
        #get date and egt opponent
        if date.find('a'):
            dateexacte=date.a.text
            #print(dateexacte)
        if opponent.find('a'):
            opponentname=opponent.a.text
        dict_infos_match=matchreport(soupmatch,dateexacte,opponentname,country)
        return dict_infos_match

In [9]:
def findallstats(linkannee):
    URLannee=DOMAIN+linkannee.attrs['href']
    #print(URLannee)
    response = rq.get(URLannee)
    soupannee=get_soup(URLannee)
    tableannee=soupannee.find('table',{'id':'matchlogs_for'})
    return tableannee

In [10]:
def iteratelines(trs):
    for tr in trs:
        #nom pays
        if findcountryandlinks(tr)!=None:
            country,links=findcountryandlinks(tr)
            #trouver liens joueurs
            if links!=None:
                for link in links:
                    table=findtableworlcupmen(link)
                    if table!=None:
                        for row in table.find_all('tr'):
                            annee,tdsannee=findsquad(row)
                            if tdsannee!=None:
                                linksyear=tdsannee.find_all('a')
                                #print(linksyear)
                                for linkannee in linksyear:
                                    #print(linkannee)
                                    tableannee=findallstats(linkannee)
                                    if tableannee!=None:
                                        print(annee)
                                        if annee>='2000':
                                            savetablecsv(tableannee,country,annee)
                                        #td: data-stat="match_report"

                                        

In [11]:
#liste = [str(i) for i in range(105,234)]
trs=soup.find_all('tr') #where th.strong.a.text>'Italy'
iteratelines(trs)

2018
2006
2002


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))